# about this notebook
두 가지 변경사항을 python script들에 반영하기 전에 시험해본다. 변경 사항은 다음과 같다.  
- img를 읽어올 때, json도 같이 읽어서 라벨 폴리곤을 그려넣는다. (B채널이 좋겠다)
- img_generation_utils 의 batch generator도 바꿔서 파일 경로도 return 하도록 한다.

In [1]:

import argparse
import json
import os
from matplotlib import pyplot as plt
import sys

import tensorflow as tf

from rz_cvae.utils import batch_generator, convert_batch_to_image_grid, read_data

# Read test_data.pickle 
test_data = read_data("./test_data")

#path = l/rz_cvae/image_data

In [3]:
len(test_data['img_paths'])

383

# 이미지 생성시 원본 경로도 출력하기
원래 이런 코드로 실행된다. 이걸 하면 이미지만 출력하는 것이 아니라 file paths도 출력되게 만들자. 그리고 지정 이미지에서 만들 수 있게 해도 좋겠는데  
```python
test_data = read_data("./test_data")
image_reconstruction(model, test_data, save_path=None) 
```

## rz_cvae/utils.py 의 함수

In [4]:
def create_image_batch(labels, model_name, use_train=False, path_return=False):
    """
    Returns the list of images corresponding to the given labels.
    """
    imgs = []
    paths = []
    imgs_id = [item[0] for item in labels]
    
    #hard code few lines...
    test_data = read_data("./test_data")
    img_paths = test_data['img_paths'][-len(test_data['test_img_ids']):] # split of test_data paths
    #img_paths = test_data['img_paths'][:-len(test_data['test_img_ids'])] # split of train_data paths
    path_cols=[0,1]

    for i in imgs_id:
        img = []
        for j in range(len(path_cols)):
            image_path = img_paths[i][j]
            paths.append(image_path)
            img.append(get_image(image_path, model_name))
        imgs.append(np.concatenate(tuple(img),axis=-1))
    if path_return=True:
        return imgs, paths
    else:
        return imgs

def get_image(image_path, model_name, img_resize = 1024):
    """
    Crops, resizes and normalizes the target image.
        - If model_name == Dense, the image is returned as a flattened numpy array with dim (64*64*3)
        - otherwise, the image is returned as a numpy array with dim (64,64,3)
    """

    img = cv2.imread(image_path)
    img = cv2.resize(img, (img_resize, img_resize))
    img = np.array(img, dtype='float32')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img /= 255.0 # Normalization to [0.,1.]

    if model_name == "Dense" :
        img = img.ravel()
    
    return img

def batch_generator(batch_dim, test_labels, model_name):
    """
    Batch generator using the given list of labels.
    """
    while True:
        batch_imgs = []
        labels = []
        for label in (test_labels):
            labels.append(label)
            if len(labels) == batch_dim:
                batch_imgs, img_paths = create_image_batch(labels, model_name, path_return=True)
                batch_labels = [x[1] for x in labels]
                yield np.asarray(batch_imgs), np.asarray(batch_labels), img_paths
                batch_imgs = []
                labels = []
                batch_labels = []
        if batch_imgs:
            yield np.asarray(batch_imgs), np.asarray(batch_labels), img_paths

## img reconstruction 함수
rz_cvae/image_generation_utils.py

In [ ]:
def image_reconstruction(model, test_data, save_path=None):
    """
    Reconstructs and plots a bacth of test images.
    """

    batch_gen, paths = batch_generator(test_data['batch_size'], test_data['test_img_ids'], model_name = 'Conv')
    images, labels= next(batch_gen)
    model_output= model((images, labels), is_train = False)
    
    print(paths)
    
    f = plt.figure(figsize=(24,60))
    ax = f.add_subplot(1,2,1)
    ax.imshow(convert_batch_to_image_grid(images))
    plt.axis('off')

    ax = f.add_subplot(1,2,2)
    ax.imshow(convert_batch_to_image_grid(model_output['recon_img'].numpy()))
    plt.axis('off')
    
    if save_path :
        plt.savefig(save_path + "reconstruction.png")

    plt.show()
    plt.clf()

    print("Reconstruction of a batch of test set images.")


In [ ]:
batch_gen = batch_generator(test_data['batch_size'], test_data['test_img_ids'], test_data['img_paths'], model_name = 'Conv')

In [5]:
len(test_data['img_paths'])

383

In [6]:
len(test_data['test_img_ids'])

20

In [11]:
td=test_data['img_paths'][-len(test_data['test_img_ids']):]

In [14]:
td

array([['sample_img/imgCrop_RI/20200928_13_RI.JPG.jpg',
        'sample_img/imgCrop_CHL/20200928_13_CHL.JPG.jpg'],
       ['sample_img/imgCrop_RI/20200928_14_RI.JPG.jpg',
        'sample_img/imgCrop_CHL/20200928_14_CHL.JPG.jpg'],
       ['sample_img/imgCrop_RI/20200928_15_RI.JPG.jpg',
        'sample_img/imgCrop_CHL/20200928_15_CHL.JPG.jpg'],
       ['sample_img/imgCrop_RI/20200928_16_RI.JPG.jpg',
        'sample_img/imgCrop_CHL/20200928_16_CHL.JPG.jpg'],
       ['sample_img/imgCrop_RI/20200929_09_RI.JPG.jpg',
        'sample_img/imgCrop_CHL/20200929_09_CHL.JPG.jpg'],
       ['sample_img/imgCrop_RI/20200929_10_RI.JPG.jpg',
        'sample_img/imgCrop_CHL/20200929_10_CHL.JPG.jpg'],
       ['sample_img/imgCrop_RI/20200929_11_RI.JPG.jpg',
        'sample_img/imgCrop_CHL/20200929_11_CHL.JPG.jpg'],
       ['sample_img/imgCrop_RI/20200929_12_RI.JPG.jpg',
        'sample_img/imgCrop_CHL/20200929_12_CHL.JPG.jpg'],
       ['sample_img/imgCrop_RI/20200929_13_RI.JPG.jpg',
        'sample_img/imgC